<a href="https://colab.research.google.com/github/reynaldovirtual/kmeans_human/blob/main/kmeans_human.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto: Implementação e Análise do Algoritmo de K-means com o Dataset Human Activity Recognition**

**Instalar e Importar Bibliotecas**

In [ ]:
# Instalar bibliotecas
!pip install numpy pandas scikit-learn matplotlib

# Importar bibliotecas essenciais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


**Obter o dataset**

In [ ]:
# Subir e descompactar o arquivo
!gdown 'https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip'
# Descompactar o arquivo zip
!unzip -o 'human+activity+recognition+using+smartphones'

Navegar pelos Arquivos

In [ ]:
import os

# Listar arquivos na pasta base
base_path = './'
for root, dirs, files in os.walk(base_path):
    print(f"Root: {root}")
    for file in files:
        print(f" - {file}")


In [ ]:
# Visualizar informações do arquivo de nomes
with open('./UCI HAR Dataset.names', 'r', encoding='latin-1') as file: # codificação alterada para 'latin-1'
    print(file.read())

In [ ]:
# Descompactar o arquivo ZIP (Dataset)
!unzip -o './UCI HAR Dataset.zip' -d './HARDataset'


In [ ]:
import os

# Listar os arquivos descompactados
for root, dirs, files in os.walk('./HARDataset'):
    print(f"Root: {root}")
    for file in files:
        print(f" - {file}")


In [ ]:
import os

# Obter caminho absoluto do arquivo
features_path = os.path.abspath('./HARDataset/UCI HAR Dataset/features.txt')
print(features_path)


Distribuição das atividades no dataset

In [ ]:
# Suponha que 'activity_labels.txt' contenha os rótulos
import pandas as pd

# Leia os rótulos de atividade do arquivo
activity_labels = pd.read_csv('./HARDataset/UCI HAR Dataset/activity_labels.txt',
                                   sep=' ', header=None,
                                   names=['activity_id', 'activity_label'])

# Converta activity_labels em um dicionário para facilitar o acesso
activity_labels = dict(zip(activity_labels['activity_id'], activity_labels['activity_label']))

print("Atividades únicas no dataset:")
print(activity_labels)

# Suponha que 'y_train.txt' contenha IDs de atividade para dados de treinamento
# Carregue os dados y_train
y_train = pd.read_csv('./HARDataset/UCI HAR Dataset/train/y_train.txt', header=None, names=['activity_id']) # carregue os dados y_train do arquivo apropriado

print("Distribuição das atividades no conjunto de treino:")
print(y_train['activity_id'].value_counts())

**Carregar os conjuntos de dados**

In [9]:
features_path = './HARDataset/UCI HAR Dataset/features.txt'
activity_labels_path = './HARDataset/UCI HAR Dataset/activity_labels.txt'
X_train_path = './HARDataset/UCI HAR Dataset/train/X_train.txt'
y_train_path = './HARDataset/UCI HAR Dataset/train/y_train.txt'
X_test_path = './HARDataset/UCI HAR Dataset/test/X_test.txt'
y_test_path = './HARDataset/UCI HAR Dataset/test/y_test.txt'


In [ ]:
# Carregar os nomes das features
features = pd.read_csv(features_path, sep='\s+', header=None, names=['index', 'feature'])
feature_names = features['feature'].values

# Confirmar sucesso
print("Features carregadas com sucesso!")
print(features.head())


In [ ]:
print("Total de features:", len(feature_names))
print("Exemplo de features:", feature_names[:10])


Dados carregados

In [ ]:
# Carregar as features e corrigir nomes duplicados
features = pd.read_csv(features_path, sep='\s+', header=None, names=['index', 'feature'])
feature_names = features['feature'].values

# Resolver duplicatas nas features
unique_feature_names = pd.Index(feature_names).to_series().reset_index()
unique_feature_names['unique_name'] = unique_feature_names[0] + "_" + (unique_feature_names.groupby(0).cumcount() + 1).astype(str)
resolved_feature_names = unique_feature_names['unique_name'].tolist()

# Carregar os dados sem usar nomes das features ainda
X_train = pd.read_csv(X_train_path, sep='\s+', header=None)
y_train = pd.read_csv(y_train_path, sep='\s+', header=None, names=['activity_id'])
X_test = pd.read_csv(X_test_path, sep='\s+', header=None)
y_test = pd.read_csv(y_test_path, sep='\s+', header=None, names=['activity_id'])

# Renomear as colunas com nomes únicos
X_train.columns = resolved_feature_names
X_test.columns = resolved_feature_names

print("Nomes únicos resolvidos e dados carregados com sucesso!")
print(features.head())


Conjunto treino e conjunto teste

In [ ]:
print("Dimensões do conjunto de treino:")
print(X_train.shape, y_train.shape)

print("\nDimensões do conjunto de teste:")
print(X_test.shape, y_test.shape)


**Normalização dos Dados**

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalizar os dados de treino e teste
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Dados normalizados!")


**Aplicação do Algoritmo K-means**

In [ ]:
from sklearn.cluster import KMeans

# Número de clusters (um por atividade)
n_clusters = 6

# Configurar o modelo K-means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Ajustar o modelo aos dados normalizados
kmeans.fit(X_train_scaled)

# Obter os rótulos dos clusters gerados
cluster_labels = kmeans.labels_
print("Clusters gerados pelo K-means:", np.unique(cluster_labels))


**Avaliar os Clusters usando o coeficiente de silhueta para avaliar a qualidade dos clusters gerados**

In [ ]:
from sklearn.metrics import silhouette_score

# Avaliação com coeficiente de silhueta
silhouette_avg = silhouette_score(X_train_scaled, cluster_labels)
print(f"Coeficiente de Silhueta: {silhouette_avg:.3f}")


**Aplicando o Elbow Method (método do cotovelo)**

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Lista para armazenar os valores de WCSS
wcss = []

# Testar para diferentes números de clusters (1 a 10, por exemplo)
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(X_train_scaled)  # Use os dados normalizados
    wcss.append(kmeans.inertia_)  # WCSS é armazenado no atributo 'inertia_'

# Plotar o gráfico do método do cotovelo
plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), wcss, marker='o')
plt.title("Método do Cotovelo")
plt.xlabel("Número de Clusters")
plt.ylabel("WCSS (Within-Cluster Sum of Squares)")
plt.grid()
plt.show()


**Visualização dos Clusters reduzindo as dimensões para 2D (com PCA) e visualizando os clusters gerados**

Verificando a separação com PCA

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduzir para 2 componentes principais
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)

# Plotar os clusters
plt.figure(figsize=(8, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=cluster_labels, cmap='viridis', s=10)
plt.title("Visualização dos Clusters (K-means)")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.colorbar(label="Cluster")
plt.show()


**Analisar as características de cada cluster formado pelo K-means**

In [ ]:
# Adicionar os rótulos reais às amostras
y_train['cluster'] = cluster_labels

# Verificar correspondência entre clusters e atividades
print(y_train.groupby(['activity_id', 'cluster']).size())


Confirmar a Presença da Coluna activity


In [ ]:
# Verificar correspondência entre atividades reais e clusters formados
activity_cluster_distribution = y_train.groupby(['activity_id', 'cluster']).size().reset_index(name='count')

# Exibir os resultados organizados
activity_cluster_distribution_sorted = activity_cluster_distribution.sort_values(by=['activity_id', 'count'], ascending=[True, False])
print(activity_cluster_distribution_sorted)


Analisar as Estatísticas das Funcionalidades em Cada Cluster

In [ ]:
# Adicionar rótulos de cluster aos dados
X_train_clustered = X_train.copy()
X_train_clustered['cluster'] = kmeans.labels_

# Calcular estatísticas das features por cluster
cluster_features_stats = X_train_clustered.groupby('cluster').mean()

# Exibir as estatísticas dos clusters
print(cluster_features_stats)


**Visualizar os Padrões de Cada Cluster**

In [ ]:
import matplotlib.pyplot as plt

# Selecionar as features mais relevantes para visualização
important_features = ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z']

# Verificar se as features importantes estão presentes nas colunas do DataFrame
for feature in important_features:
    if feature not in cluster_features_stats.columns:
        print(f"Feature '{feature}' not found in cluster_features_stats. Available columns: {cluster_features_stats.columns}")
        continue  # pule se esta feature não for encontrada

    # Plotar as médias das features por cluster
    plt.figure(figsize=(8, 5))
    plt.bar(cluster_features_stats.index, cluster_features_stats[feature])
    plt.title(f"Média de {feature} por Cluster")
    plt.xlabel("Cluster")
    plt.ylabel(f"Média de {feature}")
    plt.xticks(cluster_features_stats.index)
    plt.show()

**Comparar Clusters com Atividades Reais**

In [ ]:
# Criar uma matriz de correspondência entre atividades e clusters
activity_cluster_matrix = activity_cluster_distribution.pivot(index='activity_id', columns='cluster', values='count').fillna(0)

# Visualizar com heatmap
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.heatmap(activity_cluster_matrix, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title("Correspondência entre Atividades Reais e Clusters")
plt.xlabel("Cluster")
plt.ylabel("Atividade")
plt.show()


**Comparação com Rótulos Originais. Comparando os clusters gerados pelo K-means com os rótulos reais das atividades**

Confirmar a Presença da Coluna activity

In [ ]:
# Verificar as colunas de y_train
print(y_train.columns)

# Exemplo de dados de y_train
print(y_train.head())


Mapear Atividades

In [ ]:
# Caminho para o arquivo activity_labels.txt
activity_labels_path = './HARDataset/UCI HAR Dataset/activity_labels.txt'

# Carregar os rótulos de atividades
activity_labels = pd.read_csv(activity_labels_path, sep='\s+', header=None, names=['id', 'activity'])

# Mapear os IDs das atividades para os nomes
y_train['activity'] = y_train['activity_id'].map(activity_labels.set_index('id')['activity'])

# Verificar se a coluna foi adicionada
print(y_train.head())


Adicionando os Clusters

In [ ]:
# Adicionar os rótulos dos clusters ao DataFrame y_train
y_train['cluster'] = cluster_labels

# Verificar os dados atualizados
print(y_train.head())


Agrupar e comparar a correspondência entre clusters e atividades

In [ ]:
# Verificar correspondência entre atividades e clusters
activity_cluster_comparison = y_train.groupby(['activity', 'cluster']).size()

# Exibir os resultados
print(activity_cluster_comparison)


Organizando a tabela de forma mais legível

In [ ]:
# Converter para DataFrame para melhor visualização
activity_cluster_df = activity_cluster_comparison.reset_index(name='count')

# Ordenar por atividade e cluster
activity_cluster_df = activity_cluster_df.sort_values(by=['activity', 'cluster'])

# Exibir os primeiros resultados
print(activity_cluster_df.head())


Mapeando IDs para os Nomes das Atividades

In [ ]:
# Caminho para o arquivo activity_labels.txt
activity_labels_path = './HARDataset/UCI HAR Dataset/activity_labels.txt'

# Carregar os rótulos das atividades
activity_labels = pd.read_csv(activity_labels_path, sep='\s+', header=None, names=['id', 'activity'])

# Mapear os IDs das atividades para os nomes
y_train['activity'] = y_train['activity_id'].map(activity_labels.set_index('id')['activity'])

# Confirmar a adição da coluna 'activity'
print(y_train.head())


In [ ]:
# Verificar correspondência entre atividades reais e clusters formados
activity_cluster_distribution = y_train.groupby(['activity_id', 'cluster']).size().reset_index(name='count')

# Exibir os resultados organizados
activity_cluster_distribution_sorted = activity_cluster_distribution.sort_values(by=['activity_id', 'count'], ascending=[True, False])
print(activity_cluster_distribution_sorted)


In [ ]:
# Carregar os rótulos de atividades
activity_labels_path = './HARDataset/UCI HAR Dataset/activity_labels.txt'
activity_labels = pd.read_csv(activity_labels_path, sep='\s+', header=None, names=['id', 'activity'])
activity_cluster_df['activity_id'] = activity_cluster_df['activity'].map(activity_labels.set_index('activity')['id'])

# Substituir activity_id por activity no DataFrame
activity_cluster_df['activity'] = activity_cluster_df['activity_id'].map(activity_labels.set_index('id')['activity'])

# Criar a matriz usando os nomes das atividades
activity_cluster_matrix_named = activity_cluster_df.pivot(index='activity', columns='cluster', values='count').fillna(0)

# Plotar o heatmap com nomes das atividades
plt.figure(figsize=(10, 6))
sns.heatmap(activity_cluster_matrix_named, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title("Distribuição de Atividades por Clusters")
plt.xlabel("Cluster")
plt.ylabel("Atividade")
plt.show()
